<a href="https://colab.research.google.com/github/priyanshishrivastava18/Sentiment-Analysis/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install kaggle


Upload your kaggle.json file

In [ ]:
1# configuring the path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Importing Twiter Sentiment dataset

In [ ]:
# API to fetch the dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
  0% 0.00/80.9M [00:00<?, ?B/s]
100% 80.9M/80.9M [00:00<00:00, 860MB/s]


In [ ]:
# extracting the compressed dataser

from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


importing the dependencies

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Data Processing

In [ ]:
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

In [ ]:
twitter_data.shape

(1599999, 6)

In [ ]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
# naming the columns
column_names = ['target','id','date','flar','user','text']
twitter_data.columns = column_names
twitter_data.head()

,target,id,date,flar,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
twitter_data.shape

(1599999, 6)

In [ ]:
# misssing value in dataset
twitter_data.isnull().sum()

,0
target,0
id,0
date,0
flar,0
user,0
text,0


In [ ]:
# distribution of target column
twitter_data['target'].value_counts()

,count
target,
4,800000
0,799999


In [ ]:
twitter_data.replace({'target':{4:1}}, inplace=True)

In [ ]:
twitter_data['target'].value_counts()

,count
target,
1,800000
0,799999


0 ---> negative, 1 ---> positive

**Stemming**


stemming is the process of reducing a word to its root word

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content


In [ ]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [ ]:
twitter_data.head()

,target,id,date,flar,user,text,stemmed_content
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew,kwesidei whole crew


In [ ]:
print(twitter_data['stemmed_content'])

0          upset updat facebook text might cri result sch...
1          kenichan dive mani time ball manag save rest g...
2                            whole bodi feel itchi like fire
3                              nationwideclass behav mad see
4                                        kwesidei whole crew
                                 ...                        
1599994                           woke school best feel ever
1599995    thewdb com cool hear old walt interview http b...
1599996                         readi mojo makeov ask detail
1599997    happi th birthday boo alll time tupac amaru sh...
1599998    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1599999, dtype: object


In [ ]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599994    1
1599995    1
1599996    1
1599997    1
1599998    1
Name: target, Length: 1599999, dtype: int64


In [ ]:
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [ ]:
print(X)

['upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound'
 'whole bodi feel itchi like fire' ... 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [ ]:
print(Y)

[0 0 0 ... 1 1 1]


In [ ]:
X_train, X_test, Y_train, Y_test =train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [ ]:
print(X.shape,X_train.shape,X_test.shape)

(1599999,) (1279999,) (320000,)


In [ ]:
print(X_train)

['watch saw iv drink lil wine'
 'hire anoth employe gourmet point current hurri'
 'punish know much work tomorrow everyon els get day' ...
 'ohjustjak awkward crush nph amazingg'
 'oliveandfig wow never tweet nah yet tuesday spoil'
 'girl jenn stay twitter b c got back late amp tri relax enough sleep work amp awak']


In [ ]:
print(X_test)

['school someon slap acrosss face caus stay class total dragggg poor poor kitti'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'dad comput turn even though weird mean download song want' ...
 'destini nevertheless hooray member wonder safe trip'
 'strawberri heavi appar broke blender epic fuck fail' 'supersandro thank']


In [ ]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9453002 stored elements and shape (1279999, 461471)>
  Coords	Values
  (0, 436655)	0.27247657460069113
  (0, 354566)	0.358082129508793
  (0, 185125)	0.5296934860731303
  (0, 109298)	0.37470892870977707
  (0, 234920)	0.4196620311217523
  (0, 443035)	0.44770025018080273
  (1, 166826)	0.3977368489124923
  (1, 19762)	0.25286013616648123
  (1, 119689)	0.4309123985667578
  (1, 152015)	0.49196931362728563
  (1, 319721)	0.3128279791558143
  (1, 86730)	0.33015139495167095
  (1, 172343)	0.3785321144028949
  (2, 326288)	0.5853283798404598
  (2, 220710)	0.2626149875630722
  (2, 279017)	0.28499749936226054
  (2, 445635)	0.2356370052189603
  (2, 412554)	0.28961919890983545
  (2, 124728)	0.3317593891539697
  (2, 117914)	0.39861155079647814
  (2, 146003)	0.22130486737209085
  (2, 93826)	0.22330494345030885
  (3, 93826)	0.46202690964703447
  (3, 331805)	0.4656145237310222
  (3, 46285)	0.6887696820070169
  :	:
  (1279996, 13878)	0.43588823915

In [ ]:
print(X_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2289342 stored elements and shape (320000, 461471)>
  Coords	Values
  (0, 63768)	0.26422682589785035
  (0, 75261)	0.26552138110472423
  (0, 127038)	0.2730995033465573
  (0, 219133)	0.3295455088887744
  (0, 320624)	0.5238287544743887
  (0, 355868)	0.22363572756759686
  (0, 371332)	0.3972681545090477
  (0, 375613)	0.24658637821166524
  (0, 382367)	0.2541115193458825
  (0, 414033)	0.2624690535388268
  (1, 6452)	0.3061987253241848
  (1, 15098)	0.21044041574811612
  (1, 145408)	0.5774478727480551
  (1, 217431)	0.40260807391422926
  (1, 256686)	0.2861349119561766
  (1, 348128)	0.4737202797635419
  (1, 366259)	0.24522188688615762
  (2, 79341)	0.34512546063620464
  (2, 88422)	0.33529803711277506
  (2, 107866)	0.36857211925047617
  (2, 124443)	0.2729825984376057
  (2, 258609)	0.3072614985829275
  (2, 375890)	0.3020734075258265
  (2, 407333)	0.27060796399382786
  (2, 418804)	0.33257344872506
  :	:
  (319995, 193560)	0.373116119782695


Logistic Regression

In [ ]:
model = LogisticRegression(max_iter = 1000)

In [ ]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

Accuracy score on the training data

In [ ]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction,Y_train)

In [ ]:
print(  'Accuracy score on the training data : ',training_data_accuracy)

Accuracy score on the training data :  0.801063125830567


In [ ]:
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(X_test_prediction,Y_test)

In [ ]:
print(  'Accuracy score on the testing data : ',testing_data_accuracy)

Accuracy score on the testing data :  0.7771625


In [ ]:
import pickle

In [ ]:
filename = 'trained_model.sav'
pickle.dump(model,open(filename,'wb'))